# 元學習 (Meta Learning)

## 學習目標
- 理解「學習如何學習」的概念
- 實作 MAML (Model-Agnostic Meta-Learning)
- 學習 Few-shot Learning
- 掌握 Prototypical Networks

## 對應課程
- [李宏毅 ML 2021 - Meta Learning](https://speech.ee.ntu.edu.tw/~hylee/ml/2021-spring.php)
- [HW15: Meta Learning](https://github.com/ga642381/ML2021-Spring/tree/main/HW15)

## 元學習概述

```
傳統機器學習 vs 元學習
├── 傳統: 學習單一任務
│   訓練: 大量該任務資料
│   目標: 解決特定問題
│
└── 元學習: 學習如何學習
    訓練: 大量不同任務
    目標: 快速適應新任務

主要方法:
├── Metric-based（度量學習）
│   Siamese Networks, Prototypical Networks
├── Model-based（模型學習）
│   MANN, MetaNet
└── Optimization-based（優化學習）
    MAML, Reptile
```

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from collections import OrderedDict
import copy
import warnings
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用設備: {device}")

# 固定隨機種子
torch.manual_seed(42)
np.random.seed(42)

---
## Part 1: Few-shot Learning 設定

### N-way K-shot

- **N-way**: N 個類別
- **K-shot**: 每類 K 個訓練樣本
- **Support Set**: 用於學習的少量樣本
- **Query Set**: 用於評估的測試樣本

In [ ]:
# ========== Few-shot 資料集 ==========

class FewShotDataset(Dataset):
    """
    模擬 Few-shot 資料集
    
    生成多個任務，每個任務有 N 個類別，每類 K+Q 個樣本
    """
    
    def __init__(self, n_tasks=1000, n_classes=100, samples_per_class=20, feat_dim=64):
        self.n_tasks = n_tasks
        self.n_classes = n_classes
        self.samples_per_class = samples_per_class
        self.feat_dim = feat_dim
        
        # 生成每個類別的原型（類中心）
        self.class_prototypes = torch.randn(n_classes, feat_dim) * 2
        
        # 生成每個類別的樣本（在原型周圍）
        self.data = []
        self.labels = []
        
        for c in range(n_classes):
            samples = self.class_prototypes[c] + torch.randn(samples_per_class, feat_dim) * 0.5
            self.data.append(samples)
            self.labels.extend([c] * samples_per_class)
        
        self.data = torch.cat(self.data, dim=0)
        self.labels = torch.tensor(self.labels)
    
    def __len__(self):
        return self.n_tasks
    
    def sample_task(self, n_way, k_shot, q_query):
        """
        採樣一個 N-way K-shot 任務
        
        Returns:
            support_x: (n_way * k_shot, feat_dim)
            support_y: (n_way * k_shot,)
            query_x: (n_way * q_query, feat_dim)
            query_y: (n_way * q_query,)
        """
        # 隨機選擇 N 個類別
        selected_classes = np.random.choice(self.n_classes, n_way, replace=False)
        
        support_x, support_y = [], []
        query_x, query_y = [], []
        
        for i, c in enumerate(selected_classes):
            # 獲取該類別的所有樣本
            class_indices = (self.labels == c).nonzero().squeeze()
            perm = torch.randperm(len(class_indices))
            
            # 分割 support 和 query
            support_indices = class_indices[perm[:k_shot]]
            query_indices = class_indices[perm[k_shot:k_shot + q_query]]
            
            support_x.append(self.data[support_indices])
            support_y.extend([i] * k_shot)  # 使用相對標籤 0~N-1
            
            query_x.append(self.data[query_indices])
            query_y.extend([i] * q_query)
        
        support_x = torch.cat(support_x, dim=0)
        support_y = torch.tensor(support_y)
        query_x = torch.cat(query_x, dim=0)
        query_y = torch.tensor(query_y)
        
        return support_x, support_y, query_x, query_y

# 建立資料集
dataset = FewShotDataset(n_tasks=1000, n_classes=100, samples_per_class=20)

# 測試採樣
support_x, support_y, query_x, query_y = dataset.sample_task(n_way=5, k_shot=5, q_query=15)

print(f"5-way 5-shot 任務:")
print(f"  Support set: {support_x.shape}")
print(f"  Query set: {query_x.shape}")

---
## Part 2: Prototypical Networks

### 核心思想

1. 學習一個嵌入空間
2. 計算每個類別的「原型」（support set 樣本的均值）
3. 將 query 樣本分類到最近的原型

$$c_k = \frac{1}{|S_k|} \sum_{(x_i, y_i) \in S_k} f_\theta(x_i)$$

$$p(y=k|x) = \frac{\exp(-d(f_\theta(x), c_k))}{\sum_{k'} \exp(-d(f_\theta(x), c_{k'}))}$$

In [ ]:
# ========== Prototypical Networks ==========

class ProtoNet(nn.Module):
    """
    Prototypical Networks
    
    嵌入網路 + 原型分類
    """
    
    def __init__(self, input_dim, hidden_dim=64, output_dim=64):
        super().__init__()
        
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )
    
    def forward(self, x):
        return self.encoder(x)
    
    def compute_prototypes(self, support_x, support_y, n_way):
        """
        計算每個類別的原型
        """
        embeddings = self.forward(support_x)
        
        prototypes = []
        for c in range(n_way):
            mask = (support_y == c)
            class_embeddings = embeddings[mask]
            prototype = class_embeddings.mean(dim=0)
            prototypes.append(prototype)
        
        return torch.stack(prototypes)  # (n_way, output_dim)
    
    def classify(self, query_x, prototypes):
        """
        根據原型分類 query 樣本
        """
        query_embeddings = self.forward(query_x)  # (n_query, output_dim)
        
        # 計算歐式距離
        # query: (n_query, 1, output_dim)
        # prototypes: (1, n_way, output_dim)
        dists = torch.cdist(query_embeddings, prototypes)  # (n_query, n_way)
        
        # 負距離作為 logits
        logits = -dists
        
        return logits

# 測試
protonet = ProtoNet(input_dim=64, hidden_dim=64, output_dim=32)
print(f"ProtoNet 參數量: {sum(p.numel() for p in protonet.parameters())}")

In [ ]:
# ========== 訓練 Prototypical Networks ==========

def train_protonet(model, dataset, n_episodes=1000, n_way=5, k_shot=5, q_query=15, lr=1e-3):
    """
    訓練 Prototypical Networks
    """
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    history = {'loss': [], 'acc': []}
    
    for episode in range(n_episodes):
        # 採樣任務
        support_x, support_y, query_x, query_y = dataset.sample_task(n_way, k_shot, q_query)
        support_x, support_y = support_x.to(device), support_y.to(device)
        query_x, query_y = query_x.to(device), query_y.to(device)
        
        # 計算原型
        model.train()
        prototypes = model.compute_prototypes(support_x, support_y, n_way)
        
        # 分類 query
        logits = model.classify(query_x, prototypes)
        
        # 計算損失
        loss = F.cross_entropy(logits, query_y)
        
        # 更新
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 計算準確率
        acc = (logits.argmax(dim=1) == query_y).float().mean().item()
        
        history['loss'].append(loss.item())
        history['acc'].append(acc)
        
        if (episode + 1) % 200 == 0:
            avg_loss = np.mean(history['loss'][-100:])
            avg_acc = np.mean(history['acc'][-100:])
            print(f"Episode {episode+1}: Loss={avg_loss:.4f}, Acc={avg_acc:.4f}")
    
    return model, history

# 訓練
protonet = ProtoNet(input_dim=64, hidden_dim=64, output_dim=32)
protonet, proto_history = train_protonet(protonet, dataset, n_episodes=1000)

In [ ]:
# ========== 可視化 ==========

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 損失曲線
window = 50
smoothed_loss = np.convolve(proto_history['loss'], np.ones(window)/window, mode='valid')
smoothed_acc = np.convolve(proto_history['acc'], np.ones(window)/window, mode='valid')

axes[0].plot(smoothed_loss)
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Loss')
axes[0].set_title('ProtoNet Training Loss')
axes[0].grid(True, alpha=0.3)

axes[1].plot(smoothed_acc)
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('ProtoNet Training Accuracy')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"最終準確率: {np.mean(proto_history['acc'][-100:]):.4f}")

---
## Part 3: MAML (Model-Agnostic Meta-Learning)

### 核心思想

學習一個好的初始化參數 $\theta$，使得從這個初始化出發，只需少量梯度更新就能快速適應新任務。

### 演算法

1. **Inner Loop**: 在每個任務上用 support set 更新參數
   $$\theta'_i = \theta - \alpha \nabla_\theta L_{S_i}(\theta)$$

2. **Outer Loop**: 用 query set 上的損失更新初始參數
   $$\theta = \theta - \beta \nabla_\theta \sum_i L_{Q_i}(\theta'_i)$$

In [ ]:
# ========== MAML 模型 ==========

class MAMLModel(nn.Module):
    """
    用於 MAML 的簡單神經網路
    """
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
    
    def forward(self, x, params=None):
        """
        前向傳播
        
        如果提供 params，使用這些參數；否則使用模型自身參數
        """
        if params is None:
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
        else:
            x = F.relu(F.linear(x, params['fc1.weight'], params['fc1.bias']))
            x = F.relu(F.linear(x, params['fc2.weight'], params['fc2.bias']))
            x = F.linear(x, params['fc3.weight'], params['fc3.bias'])
        return x
    
    def get_params(self):
        """獲取參數字典"""
        return {name: param.clone() for name, param in self.named_parameters()}

# 測試
maml_model = MAMLModel(input_dim=64, hidden_dim=64, output_dim=5)
print(f"MAML 模型參數量: {sum(p.numel() for p in maml_model.parameters())}")

In [ ]:
# ========== MAML 訓練 ==========

class MAML:
    """
    MAML 演算法實現
    """
    
    def __init__(self, model, inner_lr=0.01, outer_lr=0.001, inner_steps=5):
        self.model = model.to(device)
        self.inner_lr = inner_lr
        self.outer_lr = outer_lr
        self.inner_steps = inner_steps
        
        self.meta_optimizer = optim.Adam(self.model.parameters(), lr=outer_lr)
    
    def inner_update(self, support_x, support_y, params):
        """
        Inner Loop: 在 support set 上更新參數
        """
        for _ in range(self.inner_steps):
            logits = self.model(support_x, params)
            loss = F.cross_entropy(logits, support_y)
            
            # 計算梯度
            grads = torch.autograd.grad(loss, params.values(), create_graph=True)
            
            # 更新參數
            params = {
                name: param - self.inner_lr * grad
                for (name, param), grad in zip(params.items(), grads)
            }
        
        return params
    
    def train_step(self, tasks):
        """
        一個 meta-training step
        
        Args:
            tasks: list of (support_x, support_y, query_x, query_y)
        """
        meta_loss = 0
        meta_acc = 0
        
        for support_x, support_y, query_x, query_y in tasks:
            support_x, support_y = support_x.to(device), support_y.to(device)
            query_x, query_y = query_x.to(device), query_y.to(device)
            
            # Inner Loop
            params = self.model.get_params()
            adapted_params = self.inner_update(support_x, support_y, params)
            
            # Outer Loop: 計算 query set 上的損失
            query_logits = self.model(query_x, adapted_params)
            task_loss = F.cross_entropy(query_logits, query_y)
            
            meta_loss += task_loss
            meta_acc += (query_logits.argmax(dim=1) == query_y).float().mean().item()
        
        # 更新元參數
        self.meta_optimizer.zero_grad()
        meta_loss.backward()
        self.meta_optimizer.step()
        
        return meta_loss.item() / len(tasks), meta_acc / len(tasks)

# 建立 MAML
maml_model = MAMLModel(input_dim=64, hidden_dim=64, output_dim=5)
maml = MAML(maml_model, inner_lr=0.1, outer_lr=0.001, inner_steps=3)

In [ ]:
# ========== 訓練 MAML ==========

def train_maml(maml, dataset, n_episodes=500, tasks_per_batch=4, n_way=5, k_shot=5, q_query=15):
    """
    訓練 MAML
    """
    history = {'loss': [], 'acc': []}
    
    for episode in range(n_episodes):
        # 採樣多個任務
        tasks = [
            dataset.sample_task(n_way, k_shot, q_query)
            for _ in range(tasks_per_batch)
        ]
        
        # 訓練一步
        loss, acc = maml.train_step(tasks)
        
        history['loss'].append(loss)
        history['acc'].append(acc)
        
        if (episode + 1) % 100 == 0:
            avg_loss = np.mean(history['loss'][-50:])
            avg_acc = np.mean(history['acc'][-50:])
            print(f"Episode {episode+1}: Loss={avg_loss:.4f}, Acc={avg_acc:.4f}")
    
    return history

# 訓練
maml_history = train_maml(maml, dataset, n_episodes=500, tasks_per_batch=4)

In [ ]:
# ========== 比較 MAML vs ProtoNet ==========

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

window = 20

# 損失比較
maml_loss = np.convolve(maml_history['loss'], np.ones(window)/window, mode='valid')
proto_loss = np.convolve(proto_history['loss'][:500], np.ones(window)/window, mode='valid')

axes[0].plot(proto_loss, label='ProtoNet')
axes[0].plot(maml_loss, label='MAML')
axes[0].set_xlabel('Episode')
axes[0].set_ylabel('Loss')
axes[0].set_title('Loss Comparison')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 準確率比較
maml_acc = np.convolve(maml_history['acc'], np.ones(window)/window, mode='valid')
proto_acc = np.convolve(proto_history['acc'][:500], np.ones(window)/window, mode='valid')

axes[1].plot(proto_acc, label='ProtoNet')
axes[1].plot(maml_acc, label='MAML')
axes[1].set_xlabel('Episode')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Accuracy Comparison')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n最終準確率:")
print(f"  ProtoNet: {np.mean(proto_history['acc'][-50:]):.4f}")
print(f"  MAML: {np.mean(maml_history['acc'][-50:]):.4f}")

---
## Part 4: 不同 shot 數的效果

In [ ]:
# ========== 評估不同 K-shot ==========

def evaluate_model(model, dataset, n_way, k_shots, n_episodes=100):
    """
    評估模型在不同 k-shot 設定下的表現
    """
    model.eval()
    results = {}
    
    for k in k_shots:
        accs = []
        
        for _ in range(n_episodes):
            support_x, support_y, query_x, query_y = dataset.sample_task(n_way, k, 15)
            support_x, support_y = support_x.to(device), support_y.to(device)
            query_x, query_y = query_x.to(device), query_y.to(device)
            
            with torch.no_grad():
                prototypes = model.compute_prototypes(support_x, support_y, n_way)
                logits = model.classify(query_x, prototypes)
                acc = (logits.argmax(dim=1) == query_y).float().mean().item()
                accs.append(acc)
        
        results[k] = {'mean': np.mean(accs), 'std': np.std(accs)}
        print(f"{k}-shot: {results[k]['mean']:.4f} ± {results[k]['std']:.4f}")
    
    return results

# 評估
print("ProtoNet 在不同 k-shot 下的表現:")
k_shot_results = evaluate_model(protonet, dataset, n_way=5, k_shots=[1, 3, 5, 10, 15])

In [ ]:
# ========== 可視化 k-shot 效果 ==========

k_values = list(k_shot_results.keys())
means = [k_shot_results[k]['mean'] for k in k_values]
stds = [k_shot_results[k]['std'] for k in k_values]

plt.figure(figsize=(10, 6))
plt.errorbar(k_values, means, yerr=stds, marker='o', capsize=5, linewidth=2, markersize=8)
plt.xlabel('K (number of shots)')
plt.ylabel('Accuracy')
plt.title('Few-shot Learning Performance vs Number of Shots')
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.tight_layout()
plt.show()

---
## 🎯 總結

### 元學習方法比較

| 方法 | 優點 | 缺點 |
|------|------|------|
| **ProtoNet** | 簡單、高效、穩定 | 依賴距離度量 |
| **MAML** | 通用、靈活 | 計算複雜、二階梯度 |
| **Reptile** | 簡化版 MAML | 效果略差 |

### 李宏毅 HW15 技巧

```
Prototypical Networks:
1. 使用歐式距離或餘弦距離
2. 適當的嵌入維度
3. 訓練時使用更多 way/shot

MAML:
1. 適當的 inner learning rate
2. 合適的 inner steps（通常 1-5）
3. 可以使用一階近似加速

通用技巧:
1. 資料增強
2. Task 採樣策略
3. Episode 訓練
```

### 元學習應用

```
├── 少樣本圖像分類
├── 藥物發現（少量實驗數據）
├── 機器人快速適應
├── 個性化推薦
└── 跨語言 NLP
```

### 完成！

恭喜完成所有深度學習教程！回到主 README 查看完整學習路線圖。

---
## 🏋️ 練習

### 練習 1: 比較不同 N-way 設定的影響

測試 N-way K-shot 中 N（類別數量）對分類難度的影響。

In [ ]:
# ========== 練習 1: 比較不同 N-way 設定 ==========

def evaluate_n_way(model, dataset, n_ways, k_shot=5, n_episodes=100):
    """
    評估不同 N-way 設定的效果
    """
    model.eval()
    results = {}
    
    print("練習 1: 比較不同 N-way 設定")
    print("=" * 50)
    
    for n_way in n_ways:
        accs = []
        
        for _ in range(n_episodes):
            support_x, support_y, query_x, query_y = dataset.sample_task(n_way, k_shot, 15)
            support_x, support_y = support_x.to(device), support_y.to(device)
            query_x, query_y = query_x.to(device), query_y.to(device)
            
            with torch.no_grad():
                prototypes = model.compute_prototypes(support_x, support_y, n_way)
                logits = model.classify(query_x, prototypes)
                acc = (logits.argmax(dim=1) == query_y).float().mean().item()
                accs.append(acc)
        
        results[n_way] = {'mean': np.mean(accs), 'std': np.std(accs)}
        
        # 計算隨機猜測的準確率作為參考
        random_acc = 1.0 / n_way
        print(f"{n_way:2d}-way: {results[n_way]['mean']:.4f} ± {results[n_way]['std']:.4f} "
              f"(隨機: {random_acc:.4f}, 提升: {results[n_way]['mean']/random_acc:.1f}x)")
    
    return results

# 評估不同 N-way
n_way_results = evaluate_n_way(protonet, dataset, n_ways=[2, 3, 5, 10, 15, 20], k_shot=5)

# 視覺化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

n_ways = list(n_way_results.keys())
means = [n_way_results[n]['mean'] for n in n_ways]
stds = [n_way_results[n]['std'] for n in n_ways]
random_accs = [1.0 / n for n in n_ways]

# 準確率 vs N-way
axes[0].errorbar(n_ways, means, yerr=stds, marker='o', capsize=5, linewidth=2, markersize=8, label='ProtoNet')
axes[0].plot(n_ways, random_accs, 'r--', marker='x', label='Random Guess')
axes[0].set_xlabel('N (number of ways)')
axes[0].set_ylabel('Accuracy')
axes[0].set_title('Accuracy vs Number of Classes')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 相對提升
improvements = [m / r for m, r in zip(means, random_accs)]
axes[1].bar(range(len(n_ways)), improvements, color='steelblue')
axes[1].set_xticks(range(len(n_ways)))
axes[1].set_xticklabels([f'{n}-way' for n in n_ways])
axes[1].set_ylabel('Improvement over Random')
axes[1].set_title('Relative Performance Improvement')
axes[1].axhline(y=1, color='r', linestyle='--', alpha=0.5)

for i, imp in enumerate(improvements):
    axes[1].text(i, imp + 0.1, f'{imp:.1f}x', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\n結論：")
print("- N 增加時，準確率下降（任務更難）")
print("- 但相對於隨機猜測的提升保持穩定")
print("- 這表明模型學到了有意義的表示")